# Docker Swarm

## Init Docker Swarm on Cluster

In [39]:
getent hosts cluster-00.fritz.box | awk '{ print $1 }'

192.168.56.192


In [40]:
docker node ls || docker swarm init --advertise-addr $(getent hosts cluster-00.fritz.box | awk '{ print $1 }')

Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.
Swarm initialized: current node (uyj0mb8184cxk53ldp2dp96gi) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join \
    --token SWMTKN-1-1scfli4j85q3dvhuv12iarozvrx175ti99vddcax6egus024im-40hpg6ydjdrt3g4h21lhglumy \
    192.168.56.192:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



In [41]:
JOIN_TOKEN=$(docker swarm join-token -q worker)

In [43]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

/home/pi/.ssh/id_rsa


In [44]:
for i in 01 02 03 04; do
    # Remove host key for ip
    ssh-keygen -R cluster-${i}
    # Add ip into known_hosts
    ssh-keyscan -H cluster-${i} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@cluster-${i}
done

# Host cluster-01 found: line 7 type ECDSA
# Host cluster-01 found: line 8 type RSA
# Host cluster-01 found: line 9 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed

/usr/bin/ssh-copy-id: WARNING: All keys were skipped because they already exist on the remote system.

# Host cluster-02 found: line 7 type RSA
# Host cluster-02 found: line 8 type ECDSA
# Host cluster-02 found: line 9 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-02 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-02 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-02 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
/usr/bin/ssh-co

In [45]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr $(getent hosts cluster-${i}.fritz.box | awk '{ print $1 }') $(getent hosts cluster-00.fritz.box | awk '{ print $1 }'):2377"
done

This node joined a swarm as a worker.
This node joined a swarm as a worker.
This node joined a swarm as a worker.
This node joined a swarm as a worker.


In [46]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
36r3arferwjkhkuvoohqeee9y     cluster-04          Ready               Active              
d1pcexxzd02wb0hi8ii18btj8     cluster-03          Ready               Active              
laemujt7sn7eopw271gdzzdiv     cluster-01          Ready               Active              
sb8v94q27dm7ak7d4aqpns41w     cluster-02          Ready               Active              
uyj0mb8184cxk53ldp2dp96gi *   cluster-00          Ready               Active              Leader


In [47]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  alexellis2/visualizer-arm

gpuhzkfc2fby205old38dcneg
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


## Visualizer anschauen

In [48]:
echo "http://$(getent hosts cluster-00.fritz.box | awk '{ print $1 }'):8000"

http://192.168.56.192:8000


In [49]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [50]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  stefanscherer/monitor:1.1.0

jm68lbngncrd5fxa0ag289s5x
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [51]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
jm68lbngncrd        monitor             global              5/5                 stefanscherer/monitor:1.1.0        


In [52]:
docker service create --name whoami stefanscherer/whoami:1.1.0

rzo7u50dv4hygewuflcheo47i
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [53]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
jm68lbngncrd        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
rzo7u50dv4hy        whoami              replicated          1/1                 stefanscherer/whoami:1.1.0         


In [54]:
docker service scale whoami=5

whoami scaled to 5


In [55]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
jm68lbngncrd        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
rzo7u50dv4hy        whoami              replicated          5/5                 stefanscherer/whoami:1.1.0         


In [56]:
docker service scale whoami=15

whoami scaled to 15


In [57]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
jm68lbngncrd        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
rzo7u50dv4hy        whoami              replicated          15/15               stefanscherer/whoami:1.1.0         


In [58]:
docker service scale whoami=40

whoami scaled to 40


In [59]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [60]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
jm68lbngncrd        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
rzo7u50dv4hy        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [62]:
docker service scale whoami=1

whoami scaled to 1


In [63]:
docker service scale whoami=40

whoami scaled to 40


In [64]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
jm68lbngncrd        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
rzo7u50dv4hy        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [65]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.1.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [66]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [69]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
jm68lbngncrd        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
rzo7u50dv4hy        whoami              replicated          39/40               stefanscherer/whoami:1.2.0         


# Aufräumen

In [70]:
docker service rm whoami

whoami


In [71]:
docker service rm monitor

monitor


In [72]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
36r3arferwjkhkuvoohqeee9y     cluster-04          Ready               Active              
d1pcexxzd02wb0hi8ii18btj8     cluster-03          Ready               Active              
laemujt7sn7eopw271gdzzdiv     cluster-01          Ready               Active              
sb8v94q27dm7ak7d4aqpns41w     cluster-02          Ready               Active              
uyj0mb8184cxk53ldp2dp96gi *   cluster-00          Ready               Active              Leader


In [73]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gpuhzkfc2fby        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [74]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm leave --force"
done

Node left the swarm.
Node left the swarm.
Node left the swarm.
Node left the swarm.


In [75]:
docker service rm visualizer

visualizer


In [76]:
docker swarm leave --force

Node left the swarm.
